In [1]:
!wget -q https://github.com/emanhamed/Houses-dataset/archive/refs/heads/master.zip

In [2]:
!unzip -q /content/master.zip

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip -q install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 35.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00


# mix image data with numerical/categorical data

In [5]:
import wandb
from wandb.keras import (
   WandbMetricsLogger,
   WandbModelCheckpoint,
)
run = wandb.init(project="House_price_by_image_numerical")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
config = wandb.config
wandb_callbacks = [
   WandbMetricsLogger(log_freq=5),
   WandbModelCheckpoint("models"),
]

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
import numpy as np
import argparse
import locale
import os
import pandas as pd
import glob
import cv2

def load_house_attributes(inputPath):
	# initialize the list of column names in the CSV file and then
	# load it using Pandas
	cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
	df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)
	# determine (1) the unique zip codes and (2) the number of data
	# points with each zip code
	zipcodes = df["zipcode"].value_counts().keys().tolist()
	counts = df["zipcode"].value_counts().tolist()
	# loop over each of the unique zip codes and their corresponding
	# count
	for (zipcode, count) in zip(zipcodes, counts):
		# the zip code counts for our housing dataset is *extremely*
		# unbalanced (some only having 1 or 2 houses per zip code)
		# so let's sanitize our data by removing any houses with less
		# than 25 houses per zip code
		if count < 25:
			idxs = df[df["zipcode"] == zipcode].index
			df.drop(idxs, inplace=True)
	# return the data frame
	return df

In [8]:
def process_house_attributes(df, train, test):
	# initialize the column names of the continuous data
	continuous = ["bedrooms", "bathrooms", "area"]
	# performin min-max scaling each continuous feature column to
	# the range [0, 1]
	cs = MinMaxScaler()
	trainContinuous = cs.fit_transform(train[continuous])
	testContinuous = cs.transform(test[continuous])
	# one-hot encode the zip code categorical data (by definition of
	# one-hot encoding, all output features are now in the range [0, 1])
	zipBinarizer = LabelBinarizer().fit(df["zipcode"])
	trainCategorical = zipBinarizer.transform(train["zipcode"])
	testCategorical = zipBinarizer.transform(test["zipcode"])
	# construct our training and testing data points by concatenating
	# the categorical features with the continuous features
	trainX = np.hstack([trainCategorical, trainContinuous])
	testX = np.hstack([testCategorical, testContinuous])
	# return the concatenated training and testing data
	return (trainX, testX)

In [9]:
def load_house_images(df, inputPath):
	# initialize our images array (i.e., the house images themselves)
	images = []
	# loop over the indexes of the houses
	for i in df.index.values:
		# find the four images for the house and sort the file paths,
		# ensuring the four are always in the *same order*
		basePath = os.path.sep.join([inputPath, "{}_*".format(i + 1)])
		housePaths = sorted(list(glob.glob(basePath)))
		# print(housePaths)
		# initialize our list of input images along with the output image
		# after *combining* the four input images
		inputImages = []
		outputImage = np.zeros((64, 64, 3), dtype="uint8")
		# loop over the input house paths
		for housePath in housePaths:
			# load the input image, resize it to be 32 32, and then
			# update the list of input images
			image = cv2.imread(housePath)
			image = cv2.resize(image, (32, 32))
			inputImages.append(image)
		# tile the four input images in the output image such the first
		# image goes in the top-right corner, the second image in the
		# top-left corner, the third image in the bottom-right corner,
		# and the final image in the bottom-left corner
		outputImage[0:32, 0:32] = inputImages[0]
		outputImage[0:32, 32:64] = inputImages[1]
		outputImage[32:64, 32:64] = inputImages[2]
		outputImage[32:64, 0:32] = inputImages[3]
		# add the tiled image to our set of images the network will be
		# trained on
		images.append(outputImage)
	# return our set of images
	return np.array(images)

# Create MLP

In [10]:
def create_mlp(dim, regress=False):
	# define our MLP network
	model = Sequential()
	model.add(Dense(8, input_dim=dim, activation="relu"))
	model.add(Dense(4, activation="relu"))
	# check to see if the regression node should be added
	if regress:
		model.add(Dense(1, activation="linear"))
	# return our model
	return model



# Create CNN

In [11]:
def create_cnn( regress=False):
	# initialize the input shape and channel dimension, assuming
	# TensorFlow/channels-last ordering
	inputShape = (64, 64, 3)
	# chanDim = -1
	# # define the model input
	inputs = Input(shape=inputShape)
	# # loop over the number of filters
	# for (i, f) in enumerate(filters):
	# 	# if this is the first CONV layer then set the input
	# 	# appropriately
	# 	if i == 0:
	# 		x = inputs
	# 	# CONV => RELU => BN => POOL
	# 	x = Conv2D(f, (3, 3), padding="same")(x)
	# 	x = Activation("relu")(x)
	# 	x = BatchNormalization(axis=chanDim)(x)
	# 	x = MaxPooling2D(pool_size=(2, 2))(x)

	mobile_net = tf.keras.applications.VGG16(
			include_top=False,
			weights="imagenet",
			input_shape=(64,64,3),
			pooling ='avg'
			)
	for layer in mobile_net.layers[0:-8]:
		layer.trainable = False

	# flatten the volume, then FC => RELU => BN => DROPOUT
	mobile_net = inputs
	x = Flatten()(mobile_net)
	x = Dense(16)(x)
	x = Activation("relu")(x)
	x = BatchNormalization(axis=-1)(x)
	x = Dropout(0.5)(x)
	# apply another FC layer, this one to match the number of nodes
	# coming out of the MLP
	x = Dense(4)(x)
	x = Activation("relu")(x)
	# check to see if the regression node should be added
	if regress:
		x = Dense(1, activation="linear")(x)
	# construct the CNN
	model = Model(inputs, x)
	# return the CNN
	return model

## load models and split to train and test

In [21]:
df = load_house_attributes('/content/Houses-dataset-master/Houses Dataset/HousesInfo.txt')
# load the house images and then scale the pixel intensities to the
# range [0, 1]
print("[INFO] loading house images...")
images = load_house_images(df, '/content/Houses-dataset-master/Houses Dataset')
images = images / 255.0
print("[INFO] processing data...")
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (will lead to better
# training and convergence)
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice
# process the house attributes data by performing min-max scaling
# on continuous features, one-hot encoding on categorical features,
# and then finally concatenating them together
(trainAttrX, testAttrX) = process_house_attributes(df,
												trainAttrX,
                                                testAttrX)

[INFO] loading house images...
[INFO] processing data...


## mix models

In [23]:
# create the MLP and CNN models
mlp = create_mlp(trainAttrX.shape[1], regress=False)
cnn = create_cnn(regress=False)
# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output])
# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model_mix = Model(inputs=[mlp.input, cnn.input], outputs=x)

# compile the model using mean absolute percentage error as our loss,
# implying that we seek to minimize the absolute percentage difference
# between our price *predictions* and the *actual prices*
opt = Adam(learning_rate=1e-3)
model_mix.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model

checkpoint = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/best_mix_model_house_price",
                                            save_best_only=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

print("[INFO] training model...")
model_mix.fit(
	x=[trainAttrX, trainImagesX], y=trainY,
	validation_data=([testAttrX, testImagesX], testY),
	epochs=200,
	callbacks=[stop_early, checkpoint, wandb_callbacks])

[INFO] training model...
Epoch 1/200
1/9 [==>...........................] - ETA: 15s - loss: 1056.3019

wandb: Adding directory to artifact (./models)... Done. 0.0s


9/9 [==============================] - 9s 935ms/step - loss: 389.0058 - val_loss: 9194.8926
Epoch 2/200
7/9 [======================>.......] - ETA: 0s - loss: 315.1136

wandb: Adding directory to artifact (./models)... Done. 0.0s


9/9 [==============================] - 3s 313ms/step - loss: 309.9005 - val_loss: 9459.4238
Epoch 3/200
8/9 [=========================>....] - ETA: 0s - loss: 239.0515

...

9/9 [==============================] - 2s 218ms/step - loss: 23.8407 - val_loss: 24.4204
Epoch 199/200
1/9 [==>...........................] - ETA: 0s - loss: 19.4641

wandb: Adding directory to artifact (./models)... Done. 0.0s


9/9 [==============================] - 2s 209ms/step - loss: 26.6879 - val_loss: 24.4964
Epoch 200/200
1/9 [==>...........................] - ETA: 0s - loss: 20.6493

wandb: Adding directory to artifact (./models)... Done. 0.0s


9/9 [==============================] - 2s 199ms/step - loss: 24.2357 - val_loss: 26.7448


In [25]:
# make predictions on the testing data
print("[INFO] predicting house prices...")
preds = model_mix.predict([testAttrX, testImagesX])
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
	locale.currency(df["price"].mean(), grouping=True),
	locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] predicting house prices...
3/3 [==============================] - 0s 4ms/step
[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 26.74%, std: 31.70%


In [30]:
drive.mount('/gdrive', force_remount=True)
model_mix.save('/gdrive/MyDrive/dataset/mix_House_price.h5')

Mounted at /gdrive


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
loaded_model = tf.keras.models.load_model('/gdrive/MyDrive/dataset/mix_House_price.h5')

loaded_model.evaluate(x=[testAttrX, testImagesX],y=testY)

3/3 [==============================] - 0s 10ms/step - loss: 26.7448


26.744760513305664

In [26]:
testY.values[0]*maxPrice
maxPrice

5858000

In [33]:
def predict(bedrooms, bathrooms, area, zipcode,images_path):
  bed_bath_area=abs((np.array([bedrooms,bathrooms,area])-np.array([  1.  , 1., 701.]))/(np.array([1.0e+01 ,6.5e+00 ,7.0e+03])-np.array([  1.  , 1. ,701.])))
  zipBinarizer = LabelBinarizer().fit([91901, 92276, 92677, 92880, 93446, 93510, 94501])
  inputCategorical = zipBinarizer.transform([zipcode])
  inputs = np.hstack([list(inputCategorical.ravel()), list(bed_bath_area)])
  pathimages=[]
  for image in os.listdir(images_path):
    pathh = os.path.sep.join([images_path,image])
    pathimages.append(pathh)
  pathimages.sort()
  inputImages = []
  outputImage = np.zeros((64, 64, 3), dtype="uint8")
  # loop over the input house paths
  for housePath in pathimages:
    # load the input image, resize it to be 32 32, and then
    # update the list of input images
    image = cv2.imread(housePath)
    image = cv2.resize(image, (32, 32))
    inputImages.append(image)

  outputImage[0:32, 0:32] = inputImages[0]
  outputImage[0:32, 32:64] = inputImages[1]
  outputImage[32:64, 32:64] = inputImages[2]
  outputImage[32:64, 0:32] = inputImages[3]
  num_data=inputs.reshape(1,10)
  image_data=outputImage.reshape(1,64,64,3)
  image_data = image_data / 255.0
  preds = loaded_model.predict([num_data, image_data])
  return preds
  # diff = preds.flatten() - testY
  # percentDiff = (diff / testY) * 100
  # absPercentDiff = np.abs(percentDiff)

predict(2,1,2690,92276,'/content/my_house')*5858000

1/1 [==============================] - 0s 84ms/step


array([[167210.12664586]])